In [1]:
!pip install pandas==1.0.3

     |████████████████████████████████| 10.0 MB 4.7 MB/s 
ERROR: hypertools 0.6.2 has requirement scikit-learn<0.22,>=0.19.1, but you'll have scikit-learn 0.22.2.post1 which is incompatible.
ERROR: datalab 1.1.5 has requirement pandas-profiling==1.4.0, but you'll have pandas-profiling 2.4.0 which is incompatible.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.1
    Uninstalling pandas-1.0.1:
      Successfully uninstalled pandas-1.0.1


In [2]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from math import log
import xgboost as xgb
import gc
import shap
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import seaborn as sns

n_classes = 11
folds = 5
SEED_ = 987654321

In [3]:
path = '../input/lgbm-dataset/'

group = np.load(path + 'group.npy', allow_pickle = True)
train_target = pd.read_csv('../input/liverpool-ion-switching/train.csv', usecols = ['open_channels'])
time = pd.read_csv('../input/liverpool-ion-switching/train.csv', usecols = ['open_channels', 'time'])

train = pd.read_csv(path + 'train_clean.csv')

In [4]:
def evaluate_macroF1_xgb(predictions, truth):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    labels = truth.get_label()
    pred_labels = predictions.argmax(axis = 1)
    f1 = f1_score(labels, pred_labels, average='macro')
    return ('macroF1', f1) 


params = {
    'max_depth': 12,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'learning_rate':0.1,
    'subsample':0.75,
    'colsample_bytree':0.75,
    'objective':'multi:softprob',
    'num_class': 11,
    'tree_method':'hist',
    'nthread' : -1,
}


gc.collect()

20

In [5]:
time = pd.read_csv('../input/liverpool-ion-switching/train.csv', usecols = ['open_channels', 'time'])
time['group'] = (time['time'].transform(lambda x: np.ceil(x*10000/500000)))
time['segment'] = train['segment']

In [6]:
strat =time['segment'].astype(str).copy() + time['open_channels'].astype(str).copy()
le = LabelEncoder()
strat = le.fit_transform(strat).astype(np.int16)
weight = train_target.open_channels.map(1/train_target.open_channels.value_counts())


In [7]:
del time, le
gc.collect()


40

In [8]:
gc.collect()
kf = GroupKFold(n_splits = folds)
model_list = []
score = 0
pred_oof = np.zeros((train.shape[0], n_classes))

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, strat, group)):
    print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')
    gc.collect()
    
    X_train, X_valid = train.iloc[train_index,:], train.iloc[valid_index,:]
    y_train, y_valid = train_target.iloc[train_index,:], train_target.iloc[valid_index,:]

    model = xgb.train(params =params ,dtrain = xgb.DMatrix(X_train, y_train), num_boost_round = 100000,
                  evals =[(xgb.DMatrix(X_valid, y_valid),'valid')], feval = evaluate_macroF1_xgb, maximize = True,
                  verbose_eval = 50, early_stopping_rounds =50)
          
    gc.collect()
    
    model_list += [model]
    
    valid_ = model.predict(X_valid)
    pred = valid_.argmax(axis = 1).reshape((-1))
    pred_oof[valid_index, :] = valid_

    score_temp = f1_score(y_valid, pred, average = 'macro')
    score += score_temp/folds

    del model, X_train, X_valid, y_train, y_valid
    gc.collect()
    print(f'\n\nF1_SCORE:  {score_temp}\n\n\nENDED FOLD:     {fold_n} -------\n\n\n')

np.save('pred_oof.npy', pred_oof, allow_pickle = True)

print(f'FINAL F1 SCORE:   {score}')

BEGIN FOLD:     0 -------



[0]	valid-merror:0.03315	valid-macroF1:0.93278
Multiple eval metrics have been passed: 'valid-macroF1' will be used for early stopping.

Will train until valid-macroF1 hasn't improved in 50 rounds.
[50]	valid-merror:0.03033	valid-macroF1:0.93786
Stopping. Best iteration:
[39]	valid-merror:0.03030	valid-macroF1:0.93802



TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [9]:
feature_importances = pd.DataFrame()
feature_importances['feature'] = train.columns


for fold_, mod in tqdm(enumerate(model_list)):
    imp_temp = mod.get_score(importance_type="total_gain")
    imp_temp = [imp_temp[x] for x in train.columns]

    feature_importances['fold_{}'.format(fold_ + 1)] = imp_temp
    mod.save_model(f'model{fold_}.model')

In [10]:
scaler = MinMaxScaler(feature_range=(0, 100))


feature_importances['average'] = scaler.fit_transform(X=pd.DataFrame(feature_importances[['fold_{}'.format(fold + 1) for fold in range(kf.n_splits)]].mean(axis=1)))

fig = plt.figure(figsize=(20, 16))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(50), x='average', y='feature');
plt.title('50 TOP feature importance over {} average'.format(fold_+1))


KeyError: "['fold_2', 'fold_4', 'fold_5', 'fold_3'] not in index"

In [11]:
del train, train_target
gc.collect()

32

In [12]:
test = pd.read_csv(path + 'test_clean.csv')

for i, mod in enumerate(model_list):
    if i == 0:
        pred_test = mod.predict(test)/folds
    else:
        pred_test += mod.predict(test)/folds


TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [13]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})

sub['open_channels'] = np.argmax(pred_test, axis = 1).reshape(-1)
sub.to_csv('submission.csv', index=False)

NameError: name 'pred_test' is not defined